In [ ]:
from rdkit import Chem, RDLogger
from rdkit.Chem import rdMolTransforms,rdDistGeom, rdchem,rdmolfiles,Descriptors,rdFMCS,rdmolops, MolStandardize, AllChem,rdMolDescriptors,PeriodicTable, GetPeriodicTable
from rdkit.Chem.rdchem import Mol
import pandas as pd
import numpy as np
import ast,copy,collections,itertools,os

In [ ]:
### Where reaxys file is stored
# ==================
reaxys_csv = 'datafiles/Reaxys_cc_master_noblank.csv'
# ==================

for d in ['datafiles','tempfiles','outputfiles']:
    if not os.path.isdir(d):
        os.mkdir(d)

In [ ]:
def clean_smiles(smiles:str,stereo:bool=True):
    """Santitize a SMILES"""
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles),isomericSmiles=stereo)

def clean_rxn_smiles(rxn_smiles:str):
    """Sanitize a reaction SMILES"""
    reactants,products = rxn_smiles.split('>>')
    if Chem.MolFromSmiles(reactants) is None: ### Skip over examples that RDKit cannot recognize
        return np.nan
    return f"""{'.'.join([clean_smiles(x) for x in reactants.split('.')])}>>{'.'.join([clean_smiles(x) for x in products.split('.')])}"""

def get_best_conditions(possible_rxns:list,yields:list):
    """Return index of the best reaction conditions"""
    if len(set([y for x in possible_rxns for y in x])) >1: ## If there are multiple different reactions for a RXID, check these manually
        return False
    if set([y for x in possible_rxns for y in x])==set(()):
        return 'Invalid'
    max_yields = [max(x) for x in yields]
    return max_yields.index(max(max_yields)) 

def get_condition(condition:str,index):
    """Return the best reaction conditions according to the index identified in get_best_conditions()"""
    if type(index) is int:
        return [max(x) for x in condition][index]
    else:
        return ''

def check_mass_atoms(rxn:str):
    """Return True/False if difference in reactant and product masses is < 0.1 AMU."""
    rsmiles,psmiles = rxn.split('>>')
    rmol = Chem.MolFromSmiles(rsmiles)
    pmol = Chem.MolFromSmiles(psmiles)
    if abs(Descriptors.MolWt(rmol) - Descriptors.MolWt(pmol)) >0.1:
        return False
    if len(rmol.GetAtoms())!=len(pmol.GetAtoms()):
        return False
    return True

def check_rp_bonds(rxn:str):
    """Check that the change in bonds between reactants and products reflect a Cope and Claisen rearrangement.
    Returns True if valid bond change"""
    rsmiles,psmiles = rxn.split('>>')
    rmol = Chem.MolFromSmiles(rsmiles)
    pmol = Chem.MolFromSmiles(psmiles)
    rdmolops.Kekulize(rmol,clearAromaticFlags=True)
    rbonds = [str(x.GetBondType()) for x in rmol.GetBonds() if str(x.GetBondType()) == 'DOUBLE' or str(x.GetBondType()) == 'AROMATIC' or str(x.GetBondType()) == 'TRIPLE']
    rdmolops.Kekulize(pmol,clearAromaticFlags=True)
    pbonds = [str(x.GetBondType()) for x in pmol.GetBonds() if str(x.GetBondType()) == 'DOUBLE' or str(x.GetBondType()) == 'AROMATIC' or str(x.GetBondType()) == 'TRIPLE']
    if sum([1 if x == 'DOUBLE' else 2 for x in rbonds]) != sum([1 if x == 'DOUBLE' else 2 for x in pbonds]):
        return False
    return True

def attempt_match(rxn,dic):
    try:
        return dic[rxn]
    except:
        return np.nan


# 0) Reaxys --> Valid File

In [ ]:
RDLogger.DisableLog('rdApp.*') 
### Sanitize/Canonicalize SMILES,
df_master = pd.read_csv(reaxys_csv)
df_master['Cleaned_Reaction_SmilesNOTCHECKED'] = df_master.apply (lambda row: clean_rxn_smiles(row['Reaction']), axis=1)
df_master['rxn_clean_ReaxysOrder'] = df_master.apply (lambda row: clean_rxn_smiles(row['Reaction']), axis=1)
df_master['yield_list'] = df_master.apply (lambda row: [float(x) for x in str(row['Yield (numerical)']).split(';')], axis=1)
df_master = df_master.dropna(subset=['rxn_clean_ReaxysOrder'])

df_reaxys_mp_AE_rxn_noSte_noRe = copy.copy(df_master)
df_reaxys_mp_AE_rxn_noSte = copy.copy(df_master)

### Take reaction details for reaction with highest yield per rxid
rxn_detail_dic_ls = []
for col in list(df_reaxys_mp_AE_rxn_noSte_noRe.columns.values):
    rxn_detail_dic_ls.append(df_reaxys_mp_AE_rxn_noSte_noRe.groupby('Reaction ID')[col].apply(list).to_dict())
df_rxn_details_red = pd.DataFrame(rxn_detail_dic_ls)
df_rxn_details_red

### Combine each dataentry by its RXID
rxn_details_col_dic = {x:list(df_reaxys_mp_AE_rxn_noSte_noRe.columns.values)[x] for x in range(0,df_reaxys_mp_AE_rxn_noSte_noRe.shape[1])}
df_rxn_details_red_trans = df_rxn_details_red.transpose()
df_rxn_details_red_trans = df_rxn_details_red_trans.rename(columns=rxn_details_col_dic)
df_rxn_details_red_trans = df_rxn_details_red_trans.reset_index(drop=True)

df_rxn_details_red_trans['best_conditions'] = df_rxn_details_red_trans.apply (lambda row: get_best_conditions(row['Cleaned_Reaction_SmilesNOTCHECKED'],row['yield_list']), axis=1)
df_rxn_details_red_trans['best_yield'] = df_rxn_details_red_trans.apply (lambda row: get_condition(row['yield_list'],row['best_conditions']), axis=1)
df_rxn_details_red_trans['Cleaned_Reaction_SmilesNOTCHECKED'] = df_rxn_details_red_trans.apply (lambda row: row['Cleaned_Reaction_SmilesNOTCHECKED'][0], axis=1)
df_rxn_details_red_trans['rxn_id'] = df_rxn_details_red_trans.apply (lambda row: row['Reaction ID'][0], axis=1)
df_rxn_details_red_trans.to_csv(f'tempfiles/ccMasterDetails.csv')
RDLogger.EnableLog('rdApp.*') 
df_rxn_details_red_trans

# 1) Check for Bonding Patterns in Reactants

In [ ]:
def check_templates(rxn:str,template:str):
    """Given a reaction SMILES, check that the reactant has a valid substructure (template).
    Then check that the atom economy is correct.
    Return a list of possible """
    reactants,products = rxn.split('>>')
    reactantsls = reactants.split('.')
    if len(reactants) == 1:
        return reactantsls[0]
    possible_reactants = []
    for r in reactantsls:
        if len([x for x in Chem.MolFromSmiles(r).GetSubstructMatches(Chem.MolFromSmarts(template))]) >0:
            possible_reactants.append(r)
    if len(possible_reactants) ==1:
        return possible_reactants[0]
    if len(possible_reactants) ==0:
        return False
    ### Check with product masses
    rdic = {x:Descriptors.MolWt(Chem.MolFromSmiles(x)) for x in possible_reactants}
    pdic = {x:Descriptors.MolWt(Chem.MolFromSmiles(x)) for x in products.split('.')}
    
    ### Check if reactants have any atom economy matches with the products
    prod_match_r = []
    for k,v in rdic.items():
        if len([x for x in pdic.values() if abs(v-x)<0.1]) >1:
            prod_match_r.append(k)
    ### If only 1 possible reactant
    if len(prod_match_r) ==1:
        return prod_match_r[0]
    ### If no possible reactants
    if len(prod_match_r) ==0:
        return False   
    ### If multiple possible reactants
    return 'manual'

RDLogger.DisableLog('rdApp.*') 

### Ensure that reactants are valid and capable of undergoing Cope and Claisen rearrangements
dfcc_r = copy.copy(df_rxn_details_red_trans)
all_cope_templates = '[Xx]#,=,:[Xx]-,=,:[Xx]-[Xx]-,=,:[Xx]#,=,:[Xx]'
all_cope_templates = all_cope_templates.replace('Xx','C,c,O,o,N,n,S,s,P,p') 
dfcc_r['val_reactant'] = dfcc_r.apply (lambda row: check_templates(row['Cleaned_Reaction_SmilesNOTCHECKED'],all_cope_templates), axis=1)
print(dfcc_r.shape)
dfcc_r = dfcc_r[dfcc_r['val_reactant']!=False]
print(dfcc_r.shape)
RDLogger.EnableLog('rdApp.*') 
dfcc_r.to_csv('tempfiles/dfcc_r.csv')

# 2) Check for Bonding Patterns in Products

In [ ]:
def check_prod(reactant:str,rxn:str,yields:list):
    """Check that the product is valid via atom economy and bond changes.
    If there are multiple possible products, return the one with the maximum yield
    Returns list of possible products or bool value"""
    if reactant == 'manual': ### only 5 need manual analysis
        return False
    reactants,products = rxn.split('>>')
    productls = products.split('.')
    
    ### 1) Check Masses
    rweight = Descriptors.MolWt(Chem.MolFromSmiles(reactant))
    pdic = {x:Descriptors.MolWt(Chem.MolFromSmiles(x)) for x in products.split('.')}
    prod_match_r = []
    for k,v in pdic.items():
        if  v < rweight +0.1 and v>rweight-0.1 :
            prod_match_r.append(k)
    if len(prod_match_r) ==1:
        return prod_match_r[0]
    if len(prod_match_r) ==0:
        return False

    ### 2) Check that the number of double/triple bonds are preserved
    prod_match_b = []
    rmol = Chem.MolFromSmiles(reactant)
    rdmolops.Kekulize(rmol,clearAromaticFlags=True)
    rbonds = [str(x.GetBondType()) for x in rmol.GetBonds() if str(x.GetBondType()) == 'DOUBLE' or str(x.GetBondType()) == 'TRIPLE']
    rbonds = sum([1 if x == 'DOUBLE' else 2 for x in rbonds])
    for prod in prod_match_r:
        pmol = Chem.MolFromSmiles(prod)
        rdmolops.Kekulize(pmol,clearAromaticFlags=True)
        pbonds = [str(x.GetBondType()) for x in pmol.GetBonds() if str(x.GetBondType()) == 'DOUBLE' or str(x.GetBondType()) == 'TRIPLE']
        pbonds = sum([1 if x == 'DOUBLE' else 2 for x in pbonds])
        if pbonds == rbonds:
            prod_match_b.append(prod)
    if len(prod_match_b) ==1:
        return prod_match_b[0]
    if len(prod_match_b) ==0:
        return False
    
    ### 3) Get product by yield. Do this last to eliminate non-cope/claisen reactions are not included.
    ### There will be further work to 
    if len(productls) == len(yields):
        if yields.count(max(yields))==1:
            return productls[yields.index(max(yields))]    
    
    return False 

dfcc_rprod = copy.copy(dfcc_r)
dfcc_rprod['val_product'] = dfcc_rprod.apply (lambda row: check_prod(row['val_reactant'],row['Cleaned_Reaction_SmilesNOTCHECKED'],row['yield_list'][0]), axis=1)
print(dfcc_rprod.shape)
dfcc_rprod = dfcc_rprod[dfcc_rprod['val_product'] != False]
dfcc_rprod['Cleaned_Reaction_Smiles'] = dfcc_rprod.apply (lambda row: clean_rxn_smiles(f"""{row['val_reactant']}>>{row['val_product']}"""), axis=1)
dfcc_rprod = dfcc_rprod.drop_duplicates(subset='val_reactant')
print(dfcc_rprod.shape)
dfcc_rprod.to_csv('tempfiles/ccSort.csv',index=False) ### There is a * rxn around 2000 which needs manual removal

# 3) Carry Out Reaction Mapping
# 4) RXNMapping to Excel

In [ ]:
### Mapped reactions should be found in column with name: 'rxnsmilesMap'
dfcc_rp = pd.read_csv('tempfiles/ccMapped.csv')

### Perform checks on reactions to validate them
dfcc_rp = dfcc_rp[~dfcc_rp['Cleaned_Reaction_Smiles'].str.contains('\*')]
dfcc_rp['mass_atom_check'] = dfcc_rp.apply (lambda row: check_mass_atoms(row['Cleaned_Reaction_Smiles']), axis=1)
print(dfcc_rp.shape)
dfcc_rp = dfcc_rp[dfcc_rp['mass_atom_check']==True]
print(dfcc_rp.shape,'post mass_atom_check')
dfcc_rp['pibond_check'] = dfcc_rp.apply (lambda row: check_rp_bonds(row['rxnsmilesMap']), axis=1)
dfcc_rp = dfcc_rp[dfcc_rp['pibond_check']==True]
print(dfcc_rp.shape,'post pibond_check')
dfcc_rp

# 5) RC Verification
## 5.1) Functions

In [ ]:
def detect_bond_changes(rmol:Mol,pmol:Mol):
    """Return a dictionary containing the atoms changing in a reaction.
    Atoms need to be mapped."""
    atom_changes = {}
    for n,atom in enumerate(rmol.GetAtoms()):
        ratom = {f'{sorted([x.GetBeginAtom().GetAtomMapNum(),x.GetEndAtom().GetAtomMapNum()])}':str(x.GetBondType()) for x in atom.GetBonds()}
        patom = {f'{sorted([x.GetBeginAtom().GetAtomMapNum(),x.GetEndAtom().GetAtomMapNum()])}':str(x.GetBondType()) for x in pmol.GetAtomWithIdx(atom.GetAtomMapNum()).GetBonds()}
        ### if the bond types and atoms involved in bonds change for an atom, record this in a dictionary of atom changes
        if sorted(ratom.keys()) != sorted(patom.keys()):
            atom_changes[pmol.GetAtomWithIdx(atom.GetAtomMapNum()).GetAtomMapNum()] = patom
            continue
        if sorted(ratom.values()) != sorted(patom.values()):
            atom_changes[pmol.GetAtomWithIdx(atom.GetAtomMapNum()).GetAtomMapNum()] = patom
            continue
    return atom_changes

def identify_RC_overlap(rmol:Mol,pmol:Mol,atom_changes:dict,substructSMARTS:str='[C,c]=[C,c]-[C,c]-[C,c]-[C,c]=[C,c]',overlapnum:int=3):
    """Identify reaction centres present in the reacants and those that
    have a significant overlap with the changing atoms in the reaction.
    Returns list of reaction centres"""
    possible_rcs = [[y for y in x] for x in rmol.GetSubstructMatches(Chem.MolFromSmarts(substructSMARTS))] ### pmol indexes

    ### Ensure that these reaction centre atoms are actually bonded to each other 
    valid_possible_rcs = [] 
    for rc in possible_rcs: 
        rc_bond_list = [rmol.GetBondBetweenAtoms(rc[x],rc[x-5]) for x in range(0,5)] 
        if None in rc_bond_list:
            rc_bond_list.remove(None)
        if len(rc_bond_list) == 5: ### Check there are 5 bonds between the 6 atoms
            valid_possible_rcs.append(rc)
    valid_possible_rcs = [[rmol.GetAtomWithIdx(y).GetAtomMapNum() for y in x] for x in valid_possible_rcs]
    
    ### Keep reaction centres who have a sufficient number of atoms that are undergoign changes throughout the reaciton
    overlap_rc = {','.join([str(y) for y in x]):len(set(x)-set(list(atom_changes.keys()))) for x in valid_possible_rcs}
    return [[int(x) for x in k.split(',')] for k,v in overlap_rc.items() if v == min(overlap_rc.values()) and v <=overlapnum]

def get_likely_RCs(rxn:str,rxn_templates:str,overlapnum:int=3):
    """Taking a list of possible reaction centre bonding patterns, identify and verify these reaction centres.
    Return a list containing these different reaction centres or a string if there are none"""
    rsmiles,psmiles = rxn.split('>>')
    rmol = Chem.MolFromSmiles(rsmiles)
    pmol = Chem.MolFromSmiles(psmiles)
    atom_changes = detect_bond_changes(rmol,pmol)
    reaction_centres = identify_RC_overlap(rmol,pmol,atom_changes,substructSMARTS=rxn_templates,overlapnum=overlapnum)
    final_rc = {} ### remove repeats that are ordered in reverse: 1,2,3 == 3,2,1
    for rc in reaction_centres:
        if sorted(rc) not in list(final_rc.keys()):
            final_rc[','.join([str(x) for x in sorted(rc)])] = rc
    if len(final_rc)>0:
        return list(final_rc.values())
    else:
        return 'No options'

def getonlyatomchanges(rxn:str):
    """Given a mapped reaction SMILES, detect the atoms that are undergoing bonding chagnes through the reaction.
    Returns a list"""
    rsmiles,psmiles = rxn.split('>>')
    rmol = Chem.MolFromSmiles(rsmiles)
    pmol = Chem.MolFromSmiles(psmiles)
    return list(detect_bond_changes(rmol,pmol).keys())
 
def get_unique_rcs(rc_list):
    """Ensure reaction centres are not written in the reverse and treated as unique."""
    if type(rc_list) is str:
        return []
    new_list = []
    for rc in rc_list:
        if rc not in new_list and [x for x in reversed(rc)] not in new_list:
            new_list.append(rc)
    return new_list

## 5.2) Get Potential RCs

In [ ]:
dfcc_rp_RC = copy.copy(dfcc_rp)
all_cope_templates = '[Xx]#,=,:[Xx]-,=,:[Xx]-[Xx]-,=,:[Xx]#,=,:[Xx]'.replace('Xx','C,c,O,o,N,n,S,s,P,p')

### Get the reaction centre atoms with product indexes
dfcc_rp_RC['reaction_centre_PRODatoms'] = dfcc_rp_RC.apply (lambda row: get_likely_RCs(row['rxnsmilesMap'],all_cope_templates,overlapnum=3), axis=1)
dfcc_rp_RC['reaction_centre_PRODatoms'] = dfcc_rp_RC.apply (lambda row: get_unique_rcs(row['reaction_centre_PRODatoms']), axis=1)
dfcc_rp_RC['rc_count'] = dfcc_rp_RC.apply (lambda row: len(row['reaction_centre_PRODatoms']), axis=1)
### Get the changing atoms in the reaciton
dfcc_rp_RC['changing_atoms'] = dfcc_rp_RC.apply (lambda row: getonlyatomchanges(row['rxnsmilesMap']), axis=1)

dfcc_rp_RC.to_csv('tempfiles/dfcc_rp_RC.csv',index=False)
print('These are the number of reactions with X number of reaction centres')
print(collections.Counter(dfcc_rp_RC['rc_count']))

# 6) Tautomer Check
For these reactions, tautomers will also be generated to see if the rc_count statistics can be improved
## 6.1) Generate Tautomers

In [ ]:
def mol2connectivity(rdkitmol:Mol):
    """Return dictionary of bond_number:sorted_list_involved_atoms"""
    return {n:sorted([b.GetBeginAtom().GetIdx(),b.GetEndAtom().GetIdx()]) for n,b in enumerate(rdkitmol.GetBonds())}
        
def taut_connectivity(rxn:str,tautomers:list):
    """Check that the bond connectivity in a tautomer correctly mirrors the reported product"""
    rsmiles,psmiles = rxn.split('>>')
    pmol = Chem.MolFromSmiles(psmiles)
    pbonds = mol2connectivity(pmol)
    for x in tautomers:
        if mol2connectivity(x)!=pbonds:
            return False
    return True

dfcc_rp_RC_taut = copy.copy(dfcc_rp_RC)
tautgen = MolStandardize.rdMolStandardize.TautomerEnumerator()

### Generate tautomers
dfcc_rp_RC_taut['tautomers_mols'] = dfcc_rp_RC_taut.apply (lambda row: [x for x in tautgen.Enumerate(Chem.MolFromSmiles(row['Cleaned_Reaction_Smiles'].split('>>')[1]))], axis=1)
dfcc_rp_RC_taut['tautomers_smiles'] = dfcc_rp_RC_taut.apply (lambda row: [Chem.MolToSmiles(x) for x in row['tautomers_mols']], axis=1)
dfcc_rp_RC_taut['tautomers_connectivity'] = dfcc_rp_RC_taut.apply (lambda row: taut_connectivity(row['Cleaned_Reaction_Smiles'],row['tautomers_mols']), axis=1)
dfcc_rp_RC_taut

## 6.2) Check tautomer bonds are connected to same atoms
Should all be True

In [ ]:
print('These should all be True: ',set(dfcc_rp_RC_taut['tautomers_connectivity']))

## 6.3) Reaction
### 6.3.1) Generate Possible Reactions from Given RC

In [ ]:
def getccproduct(rxn:str,rc:list,periodic_obj):
    """Take a reaction, extract the bond and atom symbols, use this to make a reaction template,
    run reaction (after generating randomly generated SMILES and molecules to eliminate
    kekulization issues), and return list of all possible products"""
    mol = Chem.MolFromSmiles(rxn.split('>>')[0])
    ### Get bond symbols
    bondsymbols = []
    map2idxdic = {x.GetAtomMapNum():x.GetIdx() for x in mol.GetAtoms()}
    for n,atom in enumerate(rc):
        if n+1 ==len(rc):
            continue
        bondsymbols.append(str(mol.GetBondBetweenAtoms(map2idxdic[atom],map2idxdic[rc[n+1]]).GetBondType()))
    bond_dic = {'DOUBLE':'=','AROMATIC':'=','SINGLE':'-','TRIPLE':'#'}
    bondsymbols = [bond_dic[x] for x in bondsymbols] +['blank']
    ### remove aromaticity so that double bonds and aromatic bonds do not cause kekulization errors later
    rdmolops.Kekulize(mol,clearAromaticFlags=True)

    ### Get atom symbols
    atomsymbols = []
    for num in rc:
        atom = mol.GetAtomWithIdx(map2idxdic[num])
        if atom.GetIsAromatic():
            atomsymbols.append(getatomsymbol(atom,periodic_obj))
        else:
            atomsymbols.append(getatomsymbol(atom,periodic_obj)) 
            
    ### prepare atom,bond symbols for reactants and prepare these so that a reaction template can be written
    rallsymbols = [[x,bondsymbols[n]] for n,x in enumerate(atomsymbols)]
    rallsymbols = [x for y in rallsymbols for x in y][:-1]
    ### prepare atom.bond symbols for products with reaction changes
    pbonds = [rxn_bondchange_nokek(x) for x in bondsymbols]
    pallsymbols = [[x,pbonds[n]] for n,x in enumerate(atomsymbols)]
    pallsymbols = [x for y in pallsymbols for x in y][:-1]
    ### generate reactant and product templates
    rtemplate = '[{0}:1]{1}[{2}:2]{3}[{4}:3]{5}[{6}:4]{7}[{8}:5]{9}[{10}:6]'.format(*rallsymbols)
    ptemplate = '[{6}:4]{7}[{8}:5]{9}[{10}:6][{0}:1]{1}[{2}:2]{3}[{4}:3]'.format(*pallsymbols)
    rxntemplate = f'{rtemplate}>>{ptemplate}'

    ### run reaction
    rdkit_rxn = AllChem.ReactionFromSmarts(rxntemplate)
    rdkit_outputs = []
    ### generate multiple SMILES (removal of kekulization can place double bonds in wrong positions)
    mol_ls = list(set([Chem.MolToSmiles(rm_map_mol(mol),doRandom=True,canonical=True) for x in range(0,25)])) + [Chem.MolToSmiles(mol)]
    for m in mol_ls:
        mol = Chem.MolFromSmiles(m)
        rdmolops.Kekulize(mol,clearAromaticFlags=True)
        rdkit_outputs.append([x[0] for x in rdkit_rxn.RunReactants(tuple([mol]))])
    rdkit_outputs = [x for y in rdkit_outputs for x in y]
    ## rdkit_rxn does not handle chirality, so all chirality is removed for consistency during matching
    cleanedoutput = []
    for x in rdkit_outputs:
        try:
            cleanedoutput.append(clean_smiles(Chem.MolToSmiles(x),stereo=False))
        except Exception as e:
            print(e)
    return cleanedoutput

def getatomsymbol(atom,periodic_obj):
    """Return element symbol for atom object"""
    return PeriodicTable.GetElementSymbol(periodic_obj,atom.GetAtomicNum()) 

def rxn_bondchange_nokek(bond:str):
    """Return string for a bond corresponding to the reduced bond order"""
    if bond=='#':
        return '='
    elif bond=='=' or bond==':':
        return '-'
    elif bond=='-':
        return '='

def gettautcheck(rxn:str,rc:list,taut_obj,periodic_obj):
    """Get all possible products from a reaction, generate tautomerizations of the product, and
    return the tautomerized product if there is a match between these lists."""
    ### get tautomer product and verification
    rdkit_products = getccproduct(rxn,rc,periodic_obj)
    rsmiles,psmiles = rxn.split('>>')
    pmol = Chem.MolFromSmiles(rm_map_and_clean_smiles(psmiles))
    rdmolops.Kekulize(pmol,clearAromaticFlags=True)    
    mol_ls = set([Chem.MolToSmiles(pmol,doRandom=True,canonical=True) for x in range(0,10)])
    reaxys_tauts = {}
    for m in mol_ls:
        pmol = Chem.MolFromSmiles(m)
        rdmolops.Kekulize(pmol,clearAromaticFlags=True)
        ### stereomarkers must be removed unforunately as rdkit does not handle chirality here well
        reaxys_tauts.update({clean_smiles(Chem.MolToSmiles(x),stereo=False).replace('/','').replace('\\',''):Chem.MolToSmiles(x) for x in taut_obj.Enumerate(pmol)})
    reaxys_tauts = {rm_map_and_clean_smiles(k):rm_map_and_clean_smiles(v) for k,v in reaxys_tauts.items()}
    for prod in rdkit_products:
        modprod = prod.replace('/','')
        modprod = modprod.replace('\\','')
        nomapprod = rm_map_and_clean_smiles(modprod)
        if  nomapprod in reaxys_tauts.keys():
            return reaxys_tauts[nomapprod]
    return None

def rm_map_and_clean_smiles(smiles:str):
    """Remove atom mapping from a SMILES string
    return SMILES"""
    my_mol = Chem.MolFromSmiles(smiles)
    atomMap = "molAtomMapNumber"
    for atom in my_mol.GetAtoms():
        if atom.HasProp(atomMap):
            atom.ClearProp(atomMap)
    return Chem.MolToSmiles(Chem.MolFromSmiles(Chem.MolToSmiles(my_mol)))

def rm_map_mol(my_mol:Mol):
    """Remove atom mapping from a Mol object
    return Mol"""
    atomMap = "molAtomMapNumber"
    for atom in my_mol.GetAtoms():
        if atom.HasProp(atomMap):
            atom.ClearProp(atomMap)
    return my_mol

def multirc_tautcheck(rxn:str,rcls:list,taut_obj,periodic_obj):
    """In cases of multiiple possible raction centres, perform the same analysis
    as in gettautcheck() but for each reaction centre.
    return a SMILES string or None"""
    multirc_dic = {str(rc):gettautcheck(rxn,rc,taut_obj,periodic_obj) for rc in rcls}    
    multirc_dic = {k:clean_smiles(v) for k,v in multirc_dic.items() if type(v) is str}
#     taut_prod = set([x for x in multirc_dic.values() if type(x) is str])
    if len(multirc_dic.values()) == 1:
        return multirc_dic[list(multirc_dic.keys())[0]]
    if len(set(multirc_dic.values())) == 1:
        return list(multirc_dic.values())[0]
    return None
    
periodic_table = GetPeriodicTable()
tautgen = MolStandardize.rdMolStandardize.TautomerEnumerator()


#### 6.3.1.1) 1 RC

In [ ]:
### Carry out the Cope and Claisen rearrangement on reactants to verify the reaction 
dfcc_rp_RC_taut_ver = copy.copy(dfcc_rp_RC_taut)
dfcc_rp_RC_taut_ver = dfcc_rp_RC_taut_ver[dfcc_rp_RC_taut_ver['rc_count']==1]
dfcc_rp_RC_taut_ver['reaction_centre_PRODatoms'] = dfcc_rp_RC_taut_ver.apply (lambda row: row['reaction_centre_PRODatoms'][0], axis=1)
dfcc_rp_RC_taut_ver['taut_prod'] = dfcc_rp_RC_taut_ver.apply (lambda row: gettautcheck(row['rxnsmilesMap'],row['reaction_centre_PRODatoms'],tautgen,periodic_table), axis=1)
dfcc_rp_RC_taut_ver


#### 6.3.1.2) Multiple RCs

In [ ]:
### Carry out the Cope and Claisen rearrangement on reactants with multiple possible reaction centres to verify reaction
dfcc_rp_RC_taut_verMULTRC = copy.copy(dfcc_rp_RC_taut)
dfcc_rp_RC_taut_verMULTRC = dfcc_rp_RC_taut_verMULTRC[dfcc_rp_RC_taut_verMULTRC['rc_count']>1]
dfcc_rp_RC_taut_verMULTRC['taut_prod'] = dfcc_rp_RC_taut_verMULTRC.apply (lambda row: multirc_tautcheck(row['rxnsmilesMap'],row['reaction_centre_PRODatoms'],tautgen,periodic_table), axis=1)
dfcc_rp_RC_taut_verMULTRC


3462,482
#### 6.3.1.3) Combine and Analyze via Changing Atoms

In [ ]:
def get_correct_rc_viaPRODS(rc:list,taut_prod,changingatoms:list):
    """Identify the correct reaction centre by the list of possible products.
    return a list of reaction centre atoms"""
    if type(rc[0]) is not list:
        return rc
    if type(taut_prod) is str:
        rc_atoms = {str(x):len(set(x)-set(changingatoms)) for x in rc}
        if collections.Counter(rc_atoms)[min(rc_atoms.values())] >1: ### get rc with best match
            return False
        else:
            return ast.literal_eval(min(rc_atoms, key=rc_atoms.get))
    else:
        return False


### Combine the dataframes where the product was verified for reactions with 1 reaction centre and the 
### dataframe for those with multiple reaction centres
dfcc_rp_RC_taut_verMULTRC_com = pd.concat([dfcc_rp_RC_taut_ver,dfcc_rp_RC_taut_verMULTRC])
dfcc_rp_RC_taut_verMULTRC_com['taut_prod_validity'] = dfcc_rp_RC_taut_verMULTRC_com.apply (lambda row: type(row['taut_prod']) is str, axis=1)
dfcc_rp_RC_taut_verMULTRC_com = dfcc_rp_RC_taut_verMULTRC_com[dfcc_rp_RC_taut_verMULTRC_com['taut_prod_validity'] == True]

### Identify the correct reaction centre
dfcc_rp_RC_taut_verMULTRC_com['reaction_centre_PRODatoms_fixed'] = dfcc_rp_RC_taut_verMULTRC_com.apply (lambda row: get_correct_rc_viaPRODS(row['reaction_centre_PRODatoms'],row['taut_prod'],row['changing_atoms']), axis=1)
dfcc_rp_RC_taut_verMULTRC_com['reaction_centre_PRODatoms_fixed_validity'] = dfcc_rp_RC_taut_verMULTRC_com.apply (lambda row: type(row['reaction_centre_PRODatoms_fixed']) is list, axis=1)
dfcc_rp_RC_taut_verMULTRC_com = dfcc_rp_RC_taut_verMULTRC_com[dfcc_rp_RC_taut_verMULTRC_com['reaction_centre_PRODatoms_fixed_validity'] == True]

### Keep those reactions with 1 reaction centre 
rc1only = dfcc_rp_RC_taut_verMULTRC_com[dfcc_rp_RC_taut_verMULTRC_com['rc_count']==1].shape[0]
rcaftertautcheck = dfcc_rp_RC_taut_verMULTRC_com.shape[0]
print(f'There were {rc1only} rxns with 1 rc but after verifying with product generation, there are now {rcaftertautcheck} rxns verified (via rxns and 1 RC)')
dfcc_rp_RC_taut_verMULTRC_com.to_csv('outputfiles/cc_results_need_manual.csv',index=False)


## The follow IDs require manual extraction:
46479232, 42924545, 42924485, 42924542, 42924487, 42924488, 42924489, 8781326, 42924495, 42924496, 8780726, 42924535, 10473016, 42924537, 8782075, 29254684, 29254686